In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("playground/train.csv")
data.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [3]:
data.dtypes

id                    int64
age                   int64
gender               object
course               object
study_hours         float64
class_attendance    float64
internet_access      object
sleep_hours         float64
sleep_quality        object
study_method         object
facility_rating      object
exam_difficulty      object
exam_score          float64
dtype: object

In [4]:
col = data.columns
col

Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')

In [5]:
def missing(col):
    mis = {}
    for i in col:
        mis[i] = data[i].isna().sum()
    return mis

In [6]:
print(missing(col))

{'id': np.int64(0), 'age': np.int64(0), 'gender': np.int64(0), 'course': np.int64(0), 'study_hours': np.int64(0), 'class_attendance': np.int64(0), 'internet_access': np.int64(0), 'sleep_hours': np.int64(0), 'sleep_quality': np.int64(0), 'study_method': np.int64(0), 'facility_rating': np.int64(0), 'exam_difficulty': np.int64(0), 'exam_score': np.int64(0)}


In [7]:
def obj(col):
    ob = []
    for i in col:
        if data[i].dtype == "object":
            ob.append(i)
        else:
            pass
    return ob

In [8]:
o1 = obj(col)
o1

['gender',
 'course',
 'internet_access',
 'sleep_quality',
 'study_method',
 'facility_rating',
 'exam_difficulty']

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [10]:
one = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
Transform = ColumnTransformer(transformers = [("one", one, o1)] , remainder = "passthrough")

In [11]:
y = data["exam_score"] 
x = data.drop(columns=["exam_score"])  

In [12]:
len(x) , len(y)

(630000, 630000)

In [13]:
t_x = Transform.fit_transform(x)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train , x_test , y_train , y_test =  train_test_split(t_x , y , test_size = 0.2)

In [16]:
len(x_train) , len(x_test) , len(y_train) , len(y_test) 

(504000, 126000, 504000, 126000)

In [17]:
from lightgbm import LGBMRegressor
m1 = LGBMRegressor(objective = "regression" , random_state = 1)

In [18]:
grid = {
    "n_estimators": [1000, 2000],
    "learning_rate": [0.05, 0.03],
    "num_leaves": [31, 63],
    "min_child_samples": [20, 30]
}

In [19]:
from sklearn.model_selection import KFold

In [20]:
cv = KFold(n_splits=5, shuffle=True, random_state = 1)
cv

KFold(n_splits=5, random_state=1, shuffle=True)

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
best = GridSearchCV(
    estimator = m1,
    param_grid = grid,
    scoring = "neg_root_mean_squared_error",
    cv = cv,
    n_jobs = -1,
    verbose = 2
)

In [ ]:
best.fit(x_train , y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [ ]:
test = pd.read_csv("playground/test.csv")
preds = best_model.predict(test)

In [ ]:
submission = pd.DataFrame({"id": test["id"], "exam_score": preds})